In [ ]:
!pip install transformers --quiet

In [1]:
!id="18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV"; conf=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$id -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'); wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$conf&id=$id" -O pubhealth.csv && rm -rf /tmp/cookies.txt

--2022-08-25 12:33:21--  https://docs.google.com/uc?export=download&confirm=&id=18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV
Resolving docs.google.com (docs.google.com)... 142.250.203.206, 2a00:1450:401b:810::200e
Connecting to docs.google.com (docs.google.com)|142.250.203.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0k-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hu3hf9fb264l5mne6gmhhtkla9f5dc4e/1661423550000/05536367513713019519/*/18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV?e=download&uuid=a2cba134-b25d-4da1-a1bb-a764bceac670 [following]
--2022-08-25 12:33:24--  https://doc-0k-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hu3hf9fb264l5mne6gmhhtkla9f5dc4e/1661423550000/05536367513713019519/*/18E24-G2Xq-y5HfM6ZatUB_8WKL_r6HOV?e=download&uuid=a2cba134-b25d-4da1-a1bb-a764bceac670
Resolving doc-0k-34-docs.googleusercontent.com (doc-0k-34-docs.googleusercontent.com)... 142.250.75.1, 2a00:1450:40

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("../../raw/pubhealth.csv",sep=",")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

X, y = data["text"].values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 1    6306
0    3769
Name: label, dtype: int64
shape  
 (10075, 11)


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 1807.11it/s]

shapes X (10075,) y (10075,)


In [2]:
data.columns

Index(['Unnamed: 0.1', 'claim_id', 'claim', 'date_published', 'explanation',
       'fact_checkers', 'text', 'sources', 'label', 'subjects', 'Unnamed: 0'],
      dtype='object')

In [1]:
import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast

from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)


class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return csr_matrix(allembeds)


class BertHead(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.head = LogisticRegression(class_weight='auto', max_iter=10000)

    def fit(self, X=None, y=None):
        self.head.fit(X, y)


    def transform(self, X=None):
        pass
    
    def predict(self, X=None):    
        return self.head.predict(X)
    
    def predict_proba(self, X=None):    
        return self.head.predict_proba(X)



using device: cuda


In [4]:
bert = BERTEmbeddings()
X_dstil_numpy = bert.transform(X).toarray()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 630/630 [00:20<00:00, 30.83it/s]


In [5]:
import pickle

# SAVE
with open('pubhealth_BERTEmbeddings.pickle', 'wb') as handle:
    pickle.dump(X_dstil_numpy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
import pickle
# LOAD
with open('../../pickles/bertcls_embeddings/pubhealth_BERTEmbeddings.pickle', 'rb') as handle:
    X_dstil_numpy=pickle.load(handle)

In [5]:
from sklearn.metrics import log_loss,accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def log_los_nonorm(x,y,average=False):
    return log_loss(x,y,normalize=True)

scores = {
    'Accuracy': {'func': accuracy_score},
    'Balanced Accuracy': {'func': balanced_accuracy_score},
    'F1': {'func': f1_score},
    'Precision': {'func': precision_score},
    'Recall': {'func': recall_score},
    'Log-likelihood': {'func': log_los_nonorm},
    #'G-mean': {'func': geometric_mean_score}
}

for score_name, score_dict in scores.items():
    scores[score_name]["list"] = []
    scores[score_name]["lab"] = []


for fold,j in enumerate(foldids):
    train = foldids[fold][1]
    test = foldids[fold][2]
    xin, yin = X_dstil_numpy[train], np.array(y[train])
    cls = BertHead()
    
    cls.fit(xin, yin)
    y_pred = cls.predict(X_dstil_numpy[test])
    y_pred_proba = cls.predict_proba(X_dstil_numpy[test])
    
    for score_name, score_dict in scores.items():
        if score_name == "Log-likelihood":
            scorval=score_dict['func'](y[test], y_pred_proba)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
        elif score_name in ["F1","Precision","Recall"]:
            scorvaln = score_dict['func'](y[test], y_pred, average=None)
            score_dict['lab'].append(scorvaln)
            scorval = score_dict['func'](y[test], y_pred, average="weighted")
            score_dict['list'].append(scorval)
            print(score_name, scorval, scorvaln)  
        else:
            scorval=score_dict['func'](y[test], y_pred)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
    print()

Accuracy 0.8157999206034141
Balanced Accuracy 0.8047291073046512
F1 0.8160494368962884 [0.75553214 0.8522293 ]
Precision 0.8163437625778484 [0.75039246 0.85577231]
Recall 0.8157999206034141 [0.76074271 0.84871551]
Log-likelihood 0.406668256145782

Accuracy 0.8183442525312686
Balanced Accuracy 0.8026682201361155
F1 0.8176982496480985 [0.75303644 0.85633537]
Precision 0.817320078425967 [0.7660626  0.84794776]
Recall 0.8183442525312686 [0.74044586 0.86489058]
Log-likelihood 0.3920066757368861

Accuracy 0.8191742755061532
Balanced Accuracy 0.8083850082570513
F1 0.8194284106221118 [0.76007374 0.8549132 ]
Precision 0.8197310936086666 [0.75470711 0.85860525]
Recall 0.8191742755061532 [0.76551724 0.85125278]
Log-likelihood 0.3909357344116411

Accuracy 0.8209251538614255
Balanced Accuracy 0.8065455808945479
F1 0.8205082779506149 [0.75791734 0.857908  ]
Precision 0.8202104694411543 [0.76655809 0.85226917]
Recall 0.8209251538614255 [0.74946921 0.86362195]
Log-likelihood 0.39837984251249425

Accur

In [6]:
numlabels = scores["F1"]["lab"][0].shape[0]
scores["F1"]["lab"][0].shape[0] 
head = "| %-20s | %-12s |" +  numlabels * " %-10s |" 
headv = ["Score", "Average"]
headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
row=head % tuple(headv)
#print("+"*len(row))
print(row)
#print("+"*len(row))

for score_name, score_dict in sorted(scores.items()) :
    if score_name == "Log-likelihood":
        headv = [score_name, np.mean(score_dict['list'])*1, np.std(score_dict['list'])*1]
    else:
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
    
    for i in range(numlabels):
        if score_name == "Log-likelihood":
            head = "| %-20s | %3.2f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
        elif score_name in ["F1","Precision","Recall"]:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels* " %4.1f ± %3.1f |"
            vals = [v[i] for v in scores[score_name]["lab"]]
            headv.append(np.mean(vals)*100)
            headv.append(np.std(vals)*100)
        else:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
    print(head % tuple(headv))
    
#print("+"*len(row))

| Score                | Average      | Kat_1      | Kat_2      |
| Accuracy             | 81.8 ± 0.330 | -          | -          |
| Balanced Accuracy    | 80.5 ± 0.403 | -          | -          |
| F1                   | 81.8 ± 0.334 | 75.5 ± 0.5 | 85.5 ± 0.3 |
| Log-likelihood       | 0.40 ± 0.006 | -          | -          |
| Precision            | 81.8 ± 0.338 | 75.9 ± 0.6 | 85.3 ± 0.5 |
| Recall               | 81.8 ± 0.330 | 75.2 ± 1.0 | 85.7 ± 0.6 |
